In [1]:
# some of the libraries I used before were updated and became out of sync, so I had to reinstall a few things to the appropriate version
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.2-py3-none-any.whl --no-deps
!pip install -q wandb

In [2]:
!pip install --no-cache-dir --force-reinstall ipywidgets
!pip install --no-cache-dir --force-reinstall tqdm joblib

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/22/2d/9c0b76f2f9cc0ebede1b9371b6f317243028ed60b90705863d493bae622e/ipywidgets-8.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for comm>=0.1.3 from https://files.pythonhosted.org/packages/e6/75/49e5bfe642f71f272236b5b2d2691cf915a7283cc0ceda56357b61daa538/comm-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for ipython>=6.1.0 from https://files.pythonhosted.org/packages/e7/e1/f4474a7ecdb7745a820f6f6039dc43c66add40f1bcc66485607d93571af6/ipython-8.32.0-py3-none-any.whl.metadata
  Obtaining dependency information for traitlets>=4.3.1 from https://files.pythonhosted.org/packages/00/c0/8f5d070730d7836adc9c9b6408dec68c6ced86b304a9b26a14df072a6e8c/traitlets-5.14.3-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.12 from https://files.pythonhosted.org/packages/21/02/88b65cc394961a60c43c70517066b6b679738caf78506a5da7b88ffcb643

In [ ]:
import os

from tqdm.notebook import tqdm
os.environ["KERAS_BACKEND"] = "jax" # could also be 'TensorFlow'
import keras_cv #built in ml models
import keras # api for deep leanrning models
import tensorflow as tf # for ml framework
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from kaggle_secrets import UserSecretsClient
import cv2 #image procvessing
import pandas as pd # data processing and analysis

import joblib # saving and loading models mid-training

import wandb

2025-02-20 10:34:40.914337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-20 10:34:40.914396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-20 10:34:40.916069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#wandb
run = wandb.init(project="hmskaggle")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
class CFG:
    verbose = 1  
    seed = 42  # random seed -- training the model multiple times do not ressult in the same outcome 
    image_size = [400, 300]  # input images
    epochs = 100 # epoch
    batch_size= 64  # batch
    drop_remainder = True  # drop incomplete batches - don't know how it worksa
    num_classes = 6 # Number of classes in the dataset
    fold = 0 # validation fold
    class_names = ['Seizure', 'LPD', 'GPD', 'LRDA','GRDA', 'Other'] # train to specific anomalies
    label2name = dict(enumerate(class_names))
    name2label = {v:k for k, v in label2name.items()}

In [6]:
keras.utils.set_random_seed(CFG.seed)

In [7]:
BASE_PATH = "/kaggle/input/hms-harmful-brain-activity-classification"
# import data from kaggle db
SPEC_DIR = "/tmp/dataset/hms-hbac"
os.makedirs(SPEC_DIR+'/train_spectrograms', exist_ok=True)

In [8]:
# train and valid - conversion of files, .parquet is difficult to process -> .npy
df = pd.read_csv(f'{BASE_PATH}/train.csv')
df['eeg_path'] = f'{BASE_PATH}/train_eegs/'+df['eeg_id'].astype(str)+'.parquet'
df['spec_path'] = f'{BASE_PATH}/train_spectrograms/'+df['spectrogram_id'].astype(str)+'.parquet'
df['spec2_path'] = f'{SPEC_DIR}/train_spectrograms/'+df['spectrogram_id'].astype(str)+'.npy'
df['class_name'] = df.expert_consensus.copy()
df['class_label'] = df.expert_consensus.map(CFG.name2label)
display(df.head(2))

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,eeg_path,spec_path,spec2_path,class_name,class_label
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0,/kaggle/input/hms-harmful-brain-activity-class...,/kaggle/input/hms-harmful-brain-activity-class...,/tmp/dataset/hms-hbac/train_spectrograms/35373...,Seizure,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0,/kaggle/input/hms-harmful-brain-activity-class...,/kaggle/input/hms-harmful-brain-activity-class...,/tmp/dataset/hms-hbac/train_spectrograms/35373...,Seizure,0


In [9]:
def process_spec(spec_id, split="train"):
    spec_path = f"{BASE_PATH}/{split}_spectrograms/{spec_id}.parquet"
    spec = pd.read_parquet(spec_path)
    spec = spec.fillna(0).values[:, 1:].T # NaN values to 0, because mid train cannot deal with NaN values
    spec = spec.astype("float32")
    np.save(f"{SPEC_DIR}/{split}_spectrograms/{spec_id}.npy", spec)

# get spec_id-s
spec_ids = df["spectrogram_id"].unique()
#execute the process_spec function across multiple CPU cores
_ = joblib.Parallel(n_jobs=-1, backend="loky")( #n_jobs = -1 -> use all availabel CPU cores
    joblib.delayed(process_spec)(spec_id, "train")
    for spec_id in tqdm(spec_ids, total=len(spec_ids))
)

  0%|          | 0/11138 [00:00<?, ?it/s]

In [10]:
# wand access
table = wandb.Table(data=df)
run.log({'data':table})
run.finish()

In [ ]:
# dataloader, procession of .npy files to extract headers
def build_augmenter(dim=CFG.image_size):
    augmenters = [
        keras_cv.layers.MixUp(alpha=2.0),

    ]
    
    def augment(img, label):
        data = {"images":img, "labels":label}
        for augmenter in augmenters:
            data = augmenter(data, training=True)
        return data["images"], data["labels"]
    
    return augment

def build_dataset(paths, offsets=None, labels=None, batch_size=32, cache=True, #batch argument
                  decode_fn=None, augment_fn=None,
                  augment=False, repeat=True, shuffle=1024, 
                  cache_dir="", drop_remainder=False):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter()
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths, offsets) if labels is None else (paths, offsets, labels)
    
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO)
    ds = ds.cache(cache_dir) if cache else ds
    ds = ds.repeat() if repeat else ds
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    ds = ds.map(augment_fn, num_parallel_calls=AUTO) if augment else ds
    ds = ds.prefetch(AUTO)
    return ds

def build_decoder(with_labels=True, target_size=CFG.image_size, dtype=32):
    def decode_signal(path, offset=None):
        file_bytes = tf.io.read_file(path)
        sig = tf.io.decode_raw(file_bytes, tf.float32)
        sig = sig[1024//dtype:]  # Remove header tag
        sig = tf.reshape(sig, [400, -1])
        
        # extract labeled subsample from spectrgram 
        if offset is not None: 
            offset = offset // 2
            sig = sig[:, offset:offset+300]
            
            # input shape of [400, 300] (also initialized inside config)
            pad_size = tf.math.maximum(0, 300 - tf.shape(sig)[1])
            sig = tf.pad(sig, [[0, 0], [0, pad_size]])
            sig = tf.reshape(sig, [400, 300])
        
        # log 
        sig = tf.clip_by_value(sig, tf.math.exp(-4.0), tf.math.exp(8.0)) # avoid 0 in log
        sig = tf.math.log(sig)
        
        # noramlize
        sig -= tf.math.reduce_mean(sig)
        sig /= tf.math.reduce_std(sig) + 1e-6
        sig = tf.tile(sig[..., None], [1, 1, 3])
        return sig
    
    def decode_label(label):
        label = tf.one_hot(label, CFG.num_classes)
        label = tf.cast(label, tf.float32)
        label = tf.reshape(label, [CFG.num_classes])
        return label
    
    def decode_with_labels(path, offset=None, label=None):
        sig = decode_signal(path, offset)
        label = decode_label(label)
        return (sig, label)
    
    return decode_with_labels if with_labels else decode_signal


In [12]:
from sklearn.model_selection import StratifiedGroupKFold
# split dataset into 5 folds -> no patients in both the training and validation set
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

df["fold"] = -1
df.reset_index(drop=True, inplace=True)
for fold, (train_idx, valid_idx) in enumerate(
    sgkf.split(df, y=df["class_label"], groups=df["patient_id"])
):
    df.loc[valid_idx, "fold"] = fold
df.groupby(["fold", "class_name"])[["eeg_id"]].count().T
# stratified sampling to make sure that even distribution exist between all folds

fold           0                                     1                    ...  \
class_name   GPD  GRDA   LPD  LRDA Other Seizure   GPD  GRDA   LPD  LRDA  ...   
eeg_id      2050  2605  1168  3793  3462    4007  4832  4856  4463  3117  ...   

fold           3                         4                                  
class_name   LPD  LRDA Other Seizure   GPD  GRDA   LPD  LRDA Other Seizure  
eeg_id      2318  4296  3884    4787  1889  3700  4249  3002  4013    4045  

[1 rows x 30 columns]

In [13]:
# sample from full data
sample_df = df.groupby("spectrogram_id").head(1).reset_index(drop=True)
train_df = sample_df[sample_df.fold != CFG.fold]
valid_df = sample_df[sample_df.fold == CFG.fold]
print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

# train
train_paths = train_df.spec2_path.values
train_offsets = train_df.spectrogram_label_offset_seconds.values.astype(int)
train_labels = train_df.class_label.values
train_ds = build_dataset(train_paths, train_offsets, train_labels, batch_size=CFG.batch_size,
                         repeat=True, shuffle=True, augment=True, cache=True)

# valid
valid_paths = valid_df.spec2_path.values
valid_offsets = valid_df.spectrogram_label_offset_seconds.values.astype(int)
valid_labels = valid_df.class_label.values
valid_ds = build_dataset(valid_paths, valid_offsets, valid_labels, batch_size=CFG.batch_size,
                         repeat=False, shuffle=False, augment=False, cache=True)

# Num Train: 9166 | Num Valid: 1972


In [20]:
# quick visualization of data
'''
imgs, tars = next(iter(train_ds))

num_imgs = 8
plt.figure(figsize=(4*4, num_imgs//4*5))
for i in range(num_imgs):
    plt.subplot(num_imgs//4, 4, i + 1)
    img = imgs[i].numpy()[...,0]  # Adjust as per your image data format
    img -= img.min()
    img /= img.max() + 1e-4
    tar = CFG.label2name[np.argmax(tars[i].numpy())]
    plt.imshow(img)
    plt.title(f"Target: {tar}")
    plt.axis('off')
    
plt.tight_layout()
plt.show()
'''

'\nimgs, tars = next(iter(train_ds))\n\nnum_imgs = 8\nplt.figure(figsize=(4*4, num_imgs//4*5))\nfor i in range(num_imgs):\n    plt.subplot(num_imgs//4, 4, i + 1)\n    img = imgs[i].numpy()[...,0]  # Adjust as per your image data format\n    img -= img.min()\n    img /= img.max() + 1e-4\n    tar = CFG.label2name[np.argmax(tars[i].numpy())]\n    plt.imshow(img)\n    plt.title(f"Target: {tar}")\n    plt.axis(\'off\')\n    \nplt.tight_layout()\nplt.show()\n'

KL Divergence Loss: Measures how much an approximated probability distribution differs from the true distribution. The goal is to minimize this divergence during training.

Adam Optimizer: An adaptive optimization algorithm that adjusts learning rates per parameter, combining momentum and RMSprop for efficient training.

In [15]:
LOSS = keras.losses.KLDivergence()
opt = keras.optimizers.Adam(learning_rate=1e-4) # adjust learning rate, faster learning results in lower accuracy

In [16]:
# keras callback to save models
ckpt_cb = keras.callbacks.ModelCheckpoint("best_model.keras",
                                         monitor='val_loss',
                                         save_best_only=True, #save model
                                         save_weights_only=False,
                                         mode='min')

In [17]:
from wandb.keras import WandbMetricsLogger
wandb.init(project="hmskaggle") # launch wandb

wandb: Currently logged in as: mozerjanos4 (mozerjanos4-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem). Use `wandb login --relogin` to force relogin


Use MixTransformer model from KerasCV, which blends CNNs and transformers for efficient feature extraction and high accuracy. It can be replaced with other models as needed.

In [18]:
from keras import Sequential
# classifier
layer = keras_cv.models.MiTBackbone.from_preset('mit_b0')

model = Sequential([
    keras.layers.Input(shape=(400,300,3)),
    keras.layers.Resizing(224,224,interpolation="bilinear"),
    layer,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(6,activation="softmax", dtype=tf.float32)
])

# compile the model  
model.compile(optimizer=opt,
              loss=LOSS)
model.summary() # summary gives valuable info about the model about to be trained, for a last check

Attaching 'config.json' from model 'keras/mit/keras/mit_b0/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/mit/keras/mit_b0/2' to your Kaggle notebook...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 224, 224, 3)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ mi_t_backbone (MiTBackbone)     │ (None, 7, 7, 256)         │  3,319,392 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling2d        │ (None, 256)               │          0 │
│ (GlobalAveragePooling2D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_48 (Dense)                │ (None, 6)                 │      1,542 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 3,320,934 (12.67 MB)

 Trainable params: 3,320,934 (12.67 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(
    train_ds, 
    epochs=CFG.epochs,
    callbacks=[ ckpt_cb,WandbMetricsLogger()], 
    steps_per_epoch=len(train_df)//CFG.batch_size,
    validation_data=valid_ds, 
    verbose=CFG.verbose
)

Epoch 1/100


2025-02-20 10:45:25.774411: E external/xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng28{k2=3,k3=0} for conv (f32[32,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,8,8]{3,2,1,0}, f32[64,32,49,49]{3,2,1,0}), window={size=49x49 pad=48_48x48_48 rhs_reversal=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2025-02-20 10:45:26.207365: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.433131892s
Trying algorithm eng28{k2=3,k3=0} for conv (f32[32,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,8,8]{3,2,1,0}, f32[64,32,49,49]{3,2,1,0}), window={size=49x49 pad=48_48x48_48 rhs_reversal=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2025-02

143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - loss: 1.2674

wandb: ERROR Unable to log learning rate.


143/143 ━━━━━━━━━━━━━━━━━━━━ 214s 663ms/step - loss: 1.2668 - val_loss: 1.3954
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 311ms/step - loss: 1.0693 - val_loss: 1.3096
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 315ms/step - loss: 1.0254 - val_loss: 1.2726
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 316ms/step - loss: 0.9701 - val_loss: 1.2694
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 321ms/step - loss: 0.9597 - val_loss: 1.2470
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 321ms/step - loss: 0.9359 - val_loss: 1.2019
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 317ms/step - loss: 0.9140 - val_loss: 1.2062
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 316ms/step - loss: 0.8997 - val_loss: 1.2050
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 323ms/step - loss: 0.8747 - val_loss: 1.1723
Epoch 10/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 322ms/step - loss: 0.8529 - val_loss: 1.1659
Epoch 11/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 317ms/step - loss: 0.8303 - val_loss: 1.1780
Epoch 12/100
143/14